In [87]:
from bs4 import BeautifulSoup
from time import sleep
from dataprep.connector import connect

import pandas as pd
import asyncio
import math
import requests

In [88]:
def get_data(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')

    left_pane = soup.find('div', class_='recipe-layout__content-left')

    prep_time = left_pane.find('div', class_= 'recipe-facts__time').text
    serves = left_pane.find('div', class_='recipe-facts__servings').text
    ingredients = left_pane.find_all('li', class_='recipe-ingredients__item')
    ingredients = [ingredient.text for ingredient in ingredients]
    recipe = soup.find_all('li', class_='recipe-directions__step')
    recipe = [step.text for step in recipe]
    nutrients = left_pane.find_all('p', class_='recipe-nutrition__item')
    nutrients = [nutrient.text for nutrient in nutrients]

    return (prep_time, serves, ingredients, recipe, nutrients)

In [89]:
async def fetch_records(queries_to_fetch):
        results = asyncio.gather(*queries_to_fetch)
        df = pd.concat(await results)
        return df

In [95]:
async def get_reviews(recipeId):
    food_connector = connect("./food", _concurrency=150)
    INITIAL_URL = 'https://api.food.com/external/v1/recipes/{}/feed/reviews?pn=1'.format(recipeId)
    print(INITIAL_URL)
    response = requests.get(INITIAL_URL)
    total_num = int(response.json()['total'])
    total_pages = math.ceil(total_num/5)
    print('total: ', total_pages)
    result = pd.DataFrame()
    orig_batch_size = 500
    batch_size = orig_batch_size
    queries = []
    for i in range(1, total_pages + 1):
        if(batch_size == 0):
            df = await fetch_records(queries)
            result = result.append(df)
            queries = []
            batch_size = orig_batch_size
            sleep(0.5)
        query = food_connector.query('reviews', pn = str(i), recipeId=recipeId)
        queries.append(query)
        batch_size = batch_size - 1
    df = await fetch_records(queries)
    result = result.append(df)
    return result

In [91]:
async def scrape_recipe(URL, recipeId):
    response = requests.get(URL)
    page_source = response.text
    reviews = await get_reviews(recipeId)
    prep_time, serves, ingredients, recipe, nutrients = get_data(page_source)
    return (prep_time, serves, ingredients, recipe, nutrients, reviews)


In [94]:
df = pd.read_csv('result.csv')

prep_times = []
serves_s = []
ingredients_s = [] 
recipes = []
nutrients_s = []

for title, url in zip(df['title'], df['dish_url']):
    prep_time, serves, ingredients, recipe, nutrients, review_data = await scrape_recipe(url, title)
    prep_times.append(prep_time)
    serves_s.append(serves)
    ingredients_s.append(ingredients)
    recipes.append(recipe)
    nutrients_s.append(nutrients)

df['prep_time'] = prep_times
df['serves'] = serves_s
df['ingredients'] = ingredients_s
df['recipe'] = recipes
df['nutrients'] = nutrients_s

df.to_csv('complete_food.csv', index=False)

/var/folders/lw/4hhwbl4d4q1dfpb0z8l91sr40000gn/T/ipykernel_1927/3036618322.py:10: RuntimeWarning: coroutine 'get_reviews' was never awaited
  prep_time, serves, ingredients, recipe, nutrients, review_data = await scrape_recipe(url, title)


TypeError: object tuple can't be used in 'await' expression

In [96]:
await get_reviews(45809)

https://api.food.com/external/v1/recipes/45809/feed/reviews?pn=1
total:  582


/opt/anaconda3/lib/python3.9/ipaddress.py:1307: RuntimeWarning: coroutine 'fetch_records' was never awaited
  self._ip = self._ip_int_from_string(addr_str)
/opt/anaconda3/lib/python3.9/ipaddress.py:1307: RuntimeWarning: coroutine 'Connector.query' was never awaited
  self._ip = self._ip_int_from_string(addr_str)


,rating,review_text,id,memberName,owner_url,num_rating
0,0,I'd have loved to be able to read through the ...,1200343,Zurie,None,None
1,4,We really enjoyed this recipe. We changed a fe...,38516,cakebaker61,None,None
2,0,Loved this recipe. Used orange juice instead ...,38062,craftykeya,None,None
3,0,OMG! I have died and gone to heaven. I did so...,1220131,The Boopster,None,None
4,4,"A good recipe! A tad on the hot side, next tim...",33260,Suey Tee,None,None
...,...,...,...,...,...,...
1,4,My family enjoyed this recipe. I thought it wa...,105090,JandJsmom,None,None
2,4,"I thought this was good, but not as awesome as...",103385,chefdownunda,None,None
3,5,I had been searching for a good bourbon chicke...,103225,catmaiden,None,None
4,4,my hubby liked the sauce but there was very li...,100746,luckycloud29,None,None
